# National Football League 

## Stadiums. Places to enjoy and business

## 1. Business problem

Stadiums are a place that gathers millions of fans.
Therefore, we have the main places for analysis and conclusion of two main problems\
**1. For fans**\
Stadiums segmenting will help many fans, traveling with their favorite team, to understand what type of stadium will be the next or in the nearest future. It will be easy to understand: it is easy to find a suitable place to stay or not, will be a lot of venues around or not etc.

**2. For Business**\
We try to segment the stadium depending on the places that are nearby to identify areas that can be developed in a certain way (need more space for food, hotels or some others)

## 2.  Data  describe 

We will use data from network. Mainly from Wikipedia. Also we need a Forsquare API to make our main work.

## *Data_Workplace*

### sample of data

In [4]:
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim 
import json 
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
%matplotlib inline
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2020.4.5.2         |   py36h9f0ad1d_0         152 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    ca-certificates-2020.4.5.2 |       hecda079_0         147 KB  conda-forge
    ------------------------------------------------------------
                       

In [5]:
# download list of stadiums
link = 'https://en.wikipedia.org/wiki/National_Football_League'
df = pd.read_html(link)[2]
columns_title  = ['Division', 'Club', 'City', 'Stadium','Capacity', 'Coordinates', 'First season', 'Head coach']
df.columns = columns_title
df = df[pd.to_numeric(df['Capacity'], errors='coerce').notnull()]
df ['Coordinates'] = df ['Coordinates'].str.split('\ufeff').str[-1].str.strip()
df.head()

,Division,Club,City,Stadium,Capacity,Coordinates,First season,Head coach
0,East,Buffalo Bills,"Orchard Park, New York",New Era Field,71608,42.774°N 78.787°W,"1960 (AFL), 1970 (NFL)",Sean McDermott
1,East,Miami Dolphins,"Miami Gardens, Florida",Hard Rock Stadium,64767,25.958°N 80.239°W,"1966 (AFL), 1970 (NFL)",Brian Flores
2,East,New England Patriots,"Foxborough, Massachusetts",Gillette Stadium,65878,42.091°N 71.264°W,"1960 (AFL), 1970 (NFL)",Bill Belichick
3,East,New York Jets,"East Rutherford, New Jersey",MetLife Stadium[C],82500,40.814°N 74.074°W,"1960 (AFL), 1970 (NFL)",Adam Gase
4,North,Baltimore Ravens,"Baltimore, Maryland",M&T Bank Stadium,71008,39.278°N 76.623°W,1996[D],John Harbaugh


In [3]:
# create a function to extract correct coordinates
def add_coord (data):
    try:
        coord = data['Coordinates']
        data['Latitude']   = float(coord[:6])
        data['Longitude']  = -float(coord[9:15])
    except: None
    return data

In [4]:
df = df.apply(add_coord, axis=1)
df.head()

,Division,Club,City,Stadium,Capacity,Coordinates,First season,Head coach,Latitude,Longitude
0,East,Buffalo Bills,"Orchard Park, New York",New Era Field,71608,42.774°N 78.787°W,"1960 (AFL), 1970 (NFL)",Sean McDermott,42.774,-78.787
1,East,Miami Dolphins,"Miami Gardens, Florida",Hard Rock Stadium,64767,25.958°N 80.239°W,"1966 (AFL), 1970 (NFL)",Brian Flores,25.958,-80.239
2,East,New England Patriots,"Foxborough, Massachusetts",Gillette Stadium,65878,42.091°N 71.264°W,"1960 (AFL), 1970 (NFL)",Bill Belichick,42.091,-71.264
3,East,New York Jets,"East Rutherford, New Jersey",MetLife Stadium[C],82500,40.814°N 74.074°W,"1960 (AFL), 1970 (NFL)",Adam Gase,40.814,-74.074
4,North,Baltimore Ravens,"Baltimore, Maryland",M&T Bank Stadium,71008,39.278°N 76.623°W,1996[D],John Harbaugh,39.278,-76.623


In [6]:
# create a function to extract correct coordinates
def add_coord (data):
    try:
        coord = data['Coordinates']
        data['Latitude']   = float(coord[:6])
        data['Longitude']  = -float(coord[9:15])
    except: None
    return data

In [8]:
df = df.apply(add_coord, axis=1)
df.head()

,Division,Club,City,Stadium,Capacity,Coordinates,First season,Head coach,Latitude,Longitude
0,East,Buffalo Bills,"Orchard Park, New York",New Era Field,71608,42.774°N 78.787°W,"1960 (AFL), 1970 (NFL)",Sean McDermott,42.774,-78.787
1,East,Miami Dolphins,"Miami Gardens, Florida",Hard Rock Stadium,64767,25.958°N 80.239°W,"1966 (AFL), 1970 (NFL)",Brian Flores,25.958,-80.239
2,East,New England Patriots,"Foxborough, Massachusetts",Gillette Stadium,65878,42.091°N 71.264°W,"1960 (AFL), 1970 (NFL)",Bill Belichick,42.091,-71.264
3,East,New York Jets,"East Rutherford, New Jersey",MetLife Stadium[C],82500,40.814°N 74.074°W,"1960 (AFL), 1970 (NFL)",Adam Gase,40.814,-74.074
4,North,Baltimore Ravens,"Baltimore, Maryland",M&T Bank Stadium,71008,39.278°N 76.623°W,1996[D],John Harbaugh,39.278,-76.623


In [9]:
# create a function to create a map
def get_coord (data, borough):
    address = borough
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    return [latitude, longitude]


def get_category_type(row):
        try:
            categories_list = row['categories']
        except:
            categories_list = row['venue.categories']
        if len(categories_list) == 0:
            return None
        else:
            return categories_list[0]['name']
        
def create_map (data, borough, limit, radius):
    import requests
    VERSION = '20180604'
    CLIENT_ID     = '####################'
    CLIENT_SECRET = '####################
    LIMIT = limit
    radius = radius
    map_res = folium.Map(width=1000,height=700,location=get_coord(data, borough), zoom_start=4)
    location=get_coord(data, borough)

    lat_list = []
    lng_list  = []
    ven_name = []
    for lat, lng, borough in zip(data['Latitude'], data['Longitude'], data['City']):
        label = '{}'.format( borough)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [float(lat), float(lng)],
            radius=10,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_res)
        
        url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, radius, LIMIT)
        results = requests.get(url).json()
        try:
            items = results['response']['groups'][0]['items']   
            dataframe = json_normalize(items)
            filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
            dataframe_filtered = dataframe.loc[:, filtered_columns]

            dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)
            dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]
        except:
            None

        for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
            lat_list.append(lat)
            lng_list.append(lng)
            ven_name.append(label)
    t_list = np.array((lat_list, lng_list, ven_name))
    t_list = t_list.T
    data_pop = pd.DataFrame({'lat': t_list[:, 0], 'lng': t_list[:, 1], 'categories': t_list[:, 2]})


        # add popular spots to the map as blue circle markers
    for lat, lng, label in zip(data_pop.lat, data_pop.lng, data_pop.categories):
        lat = float(lat)
        lng = float(lng)
        try:
            folium.features.CircleMarker(
            [lat, lng],
            radius=7,
            popup=label,
            fill=True,
            color='yellow',
            fill_color='red',
            fill_opacity=0.6
            ).add_to(map_res)
        except:
            None
    return map_res

In [10]:
create_map(df, 'Kansas', 50, 1000)

In [11]:
def create_data_for_clustering (data, k_numb):
    import requests
    from sklearn.preprocessing import StandardScaler
    num_clusters = k_numb

    lat_list = []
    lng_list  = []
    ven_name = []
    stad_list = []
    df_test = pd.DataFrame (columns = ['Stadium', 'Capacity', 'Category', 'Count of venues'])
    VERSION = '20180604'
    CLIENT_ID = '####################'
    CLIENT_SECRET = '####################'
    LIMIT = 100
    radius = 500

    for lat, lng, stad, capacity in zip(data['Latitude'], data['Longitude'], data['Stadium'], data['Capacity']):
        url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, radius, LIMIT)
        results = requests.get(url).json()
        #print(results)
        items = results['response']['groups'][0]['items']   
        dataframe = json_normalize(items)
        filtered_columns = ['venue.name', 'venue.categories']
        dataframe_filtered = dataframe.loc[:, filtered_columns]
        dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)
        dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]  
        cnt_cat = 0
        cat_dict = {}
        for category in dataframe_filtered.categories.unique():
            if category not in cat_dict:
                cat_dict[category] = 0
            cat_dict[category] +=1
        for k, v in cat_dict.items():
            df_test = df_test.append({'Stadium':stad, 'Capacity': capacity, 'Category': k, 'Count of venues':v}, ignore_index=True)
    
    df_test['Capacity'] = pd.to_numeric(df_test['Capacity'])
    list_van = []
    for el in list(range(len(df_test['Category'].values))):
        if 'Restaurant' in df_test['Category'].values[el]:
            van = 'Restaurant'
        elif 'Hotel' in df_test['Category'].values[el]:
            van = 'Hotel'
        elif 'Motel' in df_test['Category'].values[el]:
            van = 'Hotel'  
        elif 'Bar' in df_test['Category'].values[el]:
            van = 'Bar'
        elif 'Coffee' in df_test['Category'].values[el]:
            van = 'Coffee Shop' 
        elif 'Sandwich Place' in df_test['Category'].values[el]:
            van = 'Restaurant'
        elif 'Hot Dog' in df_test['Category'].values[el]:
            van = 'Restaurant'
        elif 'Breakfast Spot' in df_test['Category'].values[el]:
            van = 'Restaurant'            
        elif 'Donut' in df_test['Category'].values[el]:
            van = 'Restaurant'  
        elif 'Snack' in df_test['Category'].values[el]:
            van = 'Restaurant'                       
        elif 'Dinner' in df_test['Category'].values[el]:
            van = 'Restaurant'                        
        elif 'club' in df_test['Category'].values[el]:
            van = 'Club'
        else: van = 'Other'
        list_van.append (van)
    
    df_test['van_cat'] =  list_van    
    #print (df_test)
    
    
    df_test = df_test.pivot_table (index=['Stadium','Capacity'], values='Count of venues', columns='van_cat', aggfunc=np.sum)
    df_test.fillna(0, inplace=True)
    df_test = df_test.reset_index()
    
    df_test[['Bar','Club','Hotel', 'Other', 'Restaurant', 'Coffee Shop']] = df_test[['Bar','Club','Hotel', 'Other', 'Restaurant', 'Coffee Shop']].div(df_test['Capacity'].values,axis=0)
    df_test = df_test[['Stadium','Bar','Club','Hotel', 'Restaurant','Coffee Shop']]

    X = df_test.values[:,1:]
    X = np.nan_to_num(X)
    cluster_dataset = StandardScaler().fit_transform(X)
    k_means = KMeans(init="k-means++", n_clusters=num_clusters, n_init=12)
    k_means.fit(cluster_dataset)
    labels = k_means.labels_
    df_test["Labels"] = labels
    return df_test

In [ ]:
df_res = create_data_for_clustering(df, 3)